In [15]:
import operator
import os, math, sys
import string
import numpy as np
import random
import time
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchtext import data
from torchtext import datasets
from torchtext import vocab
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

# Import model and model helper functions
sys.path.append("..")
from src.fasttext_utils import *
from src.fasttext import *
    
data_dir = '../data'  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
# Initialize the model with the following params
model = torch.load(os.path.join(data_dir, 'model/NN_fasttext_model.pt'))
model.eval()
with open(os.path.join(data_dir, 'model/NN_fasttext_data.pkl'), 'rb') as input:
    TEXT = pickle.load(input)

In [17]:
def predict_from_sentence(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok for tok in SocialTokenizer(lowercase=True).tokenize(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.round(torch.sigmoid(model(tensor)))
    return prediction.item()


def predict_from_batch(sentence):
    model.eval()
    indexed = [TEXT.vocab.stoi[t] for t in sentence]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.round(torch.sigmoid(model(tensor)))
    return prediction.item()

In [18]:
predict_from_sentence(model, 'Some of them Im sure are good people. Both sides')

0.0

In [ ]:
temp_list = [(predict_from_batch(x.body), x.body) for i, x in enumerate(test) if int(x.label) == 1]
print(temp_list)